In [1]:
import pandas as pd
import numpy as np

In [2]:
import json
from pandas.io.json import json_normalize

In [3]:
#read in the json file as a DataFrame
sample_json_df = pd.read_json('C:/Users/andre/Desktop/data_wrangling_json/data/world_bank_projects.json')

In [4]:
#read in the json file as a string
S_json_str =json.load((open('C:/Users/andre/Desktop/data_wrangling_json/data/world_bank_projects.json')))

In [5]:
#Question number one (Find the 10 countries with most projects)

# Initial examination of the Dataframe shows that there are four columns which could be analyzed to identify the 10 countries
# with the most projects. The dataframe below is created to isolate these columns to reduce the amount of data to work with.

country = sample_json_df[['countrycode','country_namecode','countryname','countryshortname']]

In [6]:
# The loop below will apply the value_counts function to all four columns of "country" and return the first eleven results. 
# The value_counts function sorts in descending order automatically.
# The first eleven results were returned in order to confirm with 100% certainty that there isn't a tie for tenth place.

for x in country:
    print(country[x].value_counts().head(11))   


CN    19
ID    19
VN    17
IN    16
RY    13
MA    12
BD    12
NP    12
3A    11
MZ    11
BR     9
Name: countrycode, dtype: int64
People's Republic of China!$!CN         19
Republic of Indonesia!$!ID              19
Socialist Republic of Vietnam!$!VN      17
Republic of India!$!IN                  16
Republic of Yemen!$!RY                  13
Nepal!$!NP                              12
People's Republic of Bangladesh!$!BD    12
Kingdom of Morocco!$!MA                 12
Republic of Mozambique!$!MZ             11
Africa!$!3A                             11
Islamic Republic of Pakistan!$!PK        9
Name: country_namecode, dtype: int64
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
Nepal                              12
People's Republic of Bangladesh    12
Republic of Mozambique             11
Africa           

In [7]:
# Visual inspection confirms that the four tables identify the same "top ten" countries and identify the same number of projects 
# for each country. However, I would like to confirm that the "country" is identified consistently across all four columns. 
# summary1 is a created as a data frame of the "top ten" with a numerical index that can be looped over.

summary = country['countryshortname'].value_counts().head(10)
type(summary)
summary1 = summary.to_frame().reset_index()
summary1 = summary1.drop('countryshortname',axis=1)
summary1 = summary1.rename(columns = {"index": "countryshortname"}) 

In [8]:
# duplicate entries are dropped from country. This will return the unique combinations of the four columns.
country = country.drop_duplicates()

In [9]:
# The row combinations for each of the top ten countries is returned. There are only ten rows returned and naming is consistent.
# This step confirms that the countries were consistently reported across all four columns.

for x in summary1.index:
    print(country.loc[country['countryshortname']==summary1.iloc[x,0]])

   countrycode            country_namecode            countryname  \
92          ID  Republic of Indonesia!$!ID  Republic of Indonesia   

   countryshortname  
92        Indonesia  
  countrycode                 country_namecode                 countryname  \
7          CN  People's Republic of China!$!CN  People's Republic of China   

  countryshortname  
7            China  
    countrycode                    country_namecode  \
160          VN  Socialist Republic of Vietnam!$!VN   

                       countryname countryshortname  
160  Socialist Republic of Vietnam          Vietnam  
  countrycode        country_namecode        countryname countryshortname
6          IN  Republic of India!$!IN  Republic of India            India
  countrycode        country_namecode        countryname    countryshortname
3          RY  Republic of Yemen!$!RY  Republic of Yemen  Yemen, Republic of
   countrycode                      country_namecode  \
34          BD  People's Republic of Bang

In [10]:
#The final answer to question one is:
print(summary)

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Morocco               12
Nepal                 12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64


In [12]:
# Question number two (Find the top 10 major project themes using column 'mjtheme_namecode')

# json_normalize creates a normalized dataframe from nested json string

mjtheme=json_normalize(S_json_str, 'mjtheme_namecode')
print(mjtheme)
type(mjtheme)

     code                                          name
0       8                             Human development
1      11                                              
2       1                           Economic management
3       6         Social protection and risk management
4       5                         Trade and integration
5       2                      Public sector governance
6      11  Environment and natural resources management
7       6         Social protection and risk management
8       7                   Social dev/gender/inclusion
9       7                   Social dev/gender/inclusion
10      5                         Trade and integration
11      4      Financial and private sector development
12      6         Social protection and risk management
13      6                                              
14      2                      Public sector governance
15      4      Financial and private sector development
16     11  Environment and natural resources man

pandas.core.frame.DataFrame

In [13]:
# As per the data frame above, the "name" column is missing values. Thus, creating a pivot table using pd.pivot_table with "name"
# as the index will have a row for "blanks" and will not do a proper count of each project type.
# Creating a pivot table using pd.pivot_table with "code" as the index will not show the project names, so its not user friendly.
# A dictionary linking "code" to "name" is generated to convert the "code" column to the project names.

key = mjtheme.drop_duplicates()
key = key.replace('',np.nan).dropna()
key = key.set_index('code')
key = key['name'].rename('code')
key = key.to_dict()

print(key)

{'8': 'Human development', '1': 'Economic management', '6': 'Social protection and risk management', '5': 'Trade and integration', '2': 'Public sector governance', '11': 'Environment and natural resources management', '7': 'Social dev/gender/inclusion', '4': 'Financial and private sector development', '10': 'Rural development', '9': 'Urban development', '3': 'Rule of law'}


In [14]:
# The "code" numbers are looked up in the dictinary and replaced with the project names stored in the dictionary.
# the value_counts function is applied to count and list each project name in descending order.
# the first 11 results are returned to ensure that there isn't a tie for tenth place.
# visual inspection confirms that there is not a tie for tenth place.

counter = mjtheme['code'].replace(to_replace = key, value=None).value_counts().head(11)
counter

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Rule of law                                      15
Name: code, dtype: int64

In [15]:
#The final answer for question two is:
counter.head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: code, dtype: int64

In [16]:
# Question number three (In 2. above you will notice that some entries have only the code and the name is missing. 
# Create a dataframe with the missing names filled in.)

# In order to fill in the blank entries in the names column, do the following:
# 1. set "names" column to the "code" column values
# 2. Using the dictionary created for question 2, look up and replace the "code" values with the correpsonding "name" values.
# The final answer for question three is:

mjtheme['name'] = mjtheme['code']
mjtheme['name']= mjtheme['name'].replace(to_replace = key, value=None)
mjtheme

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion
